### MY CONTACT (WHATSAPP): 0206546543

## OBJECT DETECTION WITH OPENCV

In [1]:
# import libraries
import cv2 # help: https://docs.opencv.org/4.x/d6/d00/tutorial_py_root.html

In [31]:
# check the opencv version is use
cv2.__version__

'4.6.0'

#### PART 1 - WORKING WITH IMAGES

In [10]:
# read an image
img_data = cv2.imread("images/img-1.jpg", cv2.IMREAD_COLOR)

# get the dimensions of the image
img_height, img_width, channels = img_data.shape
print(f"{img_width = }, {img_height = }, {channels = }")

# show an image
cv2.imshow("Image Color", img_data)

# wait indefinitly for a key press event
keypressed = cv2.waitKey(delay=0)
# if the ESC key is pressed, destroy all windows and release resources
if keypressed == 27:
    cv2.destroyAllWindows()

img_width = 612, img_height = 408, channels = 3


#### SOME IMAGE OPERATIONS

In [22]:
# convert image from colored to grayscale
img_data_gray = cv2.cvtColor(img_data, cv2.COLOR_BGR2GRAY)

# put text on the image
img_data_text = cv2.putText(
    img=img_data.copy(), 
    text="Welcome to OpenCV", 
    org=((img_width // 2) - 150, img_height // 2), 
    fontFace=cv2.FONT_HERSHEY_COMPLEX, 
    fontScale=1, 
    color=(0, 0, 255), 
    thickness=2, 
    lineType=cv2.LINE_AA
)

# draw a line
img_data_line = cv2.line(
    img=img_data.copy(),
    pt1=((img_width // 2) - 150, img_height // 2), 
    pt2=(300, 150), 
    color=(255, 0, 255),
    thickness=3,
    lineType=cv2.LINE_AA
)

# draw a rectangle
img_data_rect = cv2.rectangle(
    img=img_data.copy(), 
    pt1=((img_width // 2) - 150, img_height // 2), 
    pt2=(300, 150), 
    color=(255, 0, 0), 
    thickness=-1,
    lineType=cv2.LINE_AA
)

# draw a circle
img_data_circle = cv2.circle(
    img=img_data.copy(),
    center=(img_width // 2, img_height // 2),
    radius=100,
    color=(0, 255, 0), 
    thickness=3,
    lineType=cv2.LINE_AA
)

# draw an ellipse
img_data_ellipse = cv2.ellipse(
    img=img_data.copy(),
    center=(img_width // 2, img_height // 2),
    axes=(100, 150),
    angle=0,
    startAngle=20,
    endAngle=380,
    color=(0, 255, 0), 
    thickness=3,
    lineType=cv2.LINE_AA
)

# show image
# cv2.imshow("Image Gray", img_data_gray)
# cv2.imshow("Image Text", img_data_text)
cv2.imshow("Image Line", img_data_line)
cv2.imshow("Image Rectangle", img_data_rect)
cv2.imshow("Image Circle", img_data_circle)
cv2.imshow("Image Ellipse", img_data_ellipse)

# wait indefinitly for a key press event
keypressed = cv2.waitKey(0)
# if the ESC key is pressed, destroy all windows and release resources
if keypressed == 27:
    cv2.destroyAllWindows()

#### PART 2 - OBJECT DETECTION IN IMAGES

In [23]:
# load classifiers (source: https://github.com/opencv/opencv/tree/3.4/data)
face_cascade = cv2.CascadeClassifier()
face_cascade.load(cv2.samples.findFile("haarcascades/haarcascade_frontalface_alt.xml"))
eyes_cascade = cv2.CascadeClassifier()
eyes_cascade.load(cv2.samples.findFile("haarcascades/haarcascade_eye_tree_eyeglasses.xml"))
face_cascade_cuda = cv2.CascadeClassifier()
face_cascade_cuda.load(cv2.samples.findFile("haarcascades_cuda/haarcascade_frontalface_alt.xml"))
eyes_cascade_cuda = cv2.CascadeClassifier()
eyes_cascade_cuda.load(cv2.samples.findFile("haarcascades_cuda/haarcascade_eye_tree_eyeglasses.xml"))

# check whether the classifiers were loaded or not
if not face_cascade:
    print("Failed to load the face cascade")
if not eyes_cascade:
    print("Failed to load the eye cascade")
if not face_cascade_cuda:
    print("Failed to load the face cascade for CUDA")
if not eyes_cascade_cuda:
    print("Failed to load the eye cascade for CUDA")

In [25]:
def detect_face(frame):
    """Detect face

    Args:
        frame (Mat): An image matrix or frame

    Returns:
        Mat: The processed frame
    """
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.equalizeHist(frame_gray)
    #-- Detect faces
    faces = face_cascade.detectMultiScale(frame_gray)
    for (x,y,w,h) in faces:
        center = (x + w // 2, y + h // 2)
        frame = cv2.ellipse(frame, center, (w // 2, h // 2), 0, 0, 360, (255, 0, 255), 4)
        faceROI = frame_gray[y:y+h, x:x+w]
        #-- In each face, detect eyes
        eyes = eyes_cascade.detectMultiScale(faceROI)
        for (x2,y2,w2,h2) in eyes:
            eye_center = (x + x2 + w2 // 2, y + y2 + h2 // 2)
            radius = int(round((w2 + h2) * 0.25))
            frame = cv2.circle(frame, eye_center, radius, (255, 0, 0 ), 4)
    return frame


def detect_face_cuda(frame):
    """Detect face using CUDA

    Args:
        frame (Mat): An image matrix or frame

    Returns:
        Mat: The processed frame
    """
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.equalizeHist(frame_gray)
    #-- Detect faces
    faces = face_cascade_cuda.detectMultiScale(frame_gray)
    for (x,y,w,h) in faces:
        center = (x + w // 2, y + h // 2)
        frame = cv2.ellipse(frame, center, (w // 2, h // 2), 0, 0, 360, (255, 0, 255), 4)
        faceROI = frame_gray[y:y+h,x:x+w]
        #-- In each face, detect eyes
        eyes = eyes_cascade_cuda.detectMultiScale(faceROI)
        for (x2,y2,w2,h2) in eyes:
            eye_center = (x + x2 + w2 // 2, y + y2 + h2 // 2)
            radius = int(round((w2 + h2) * 0.25))
            frame = cv2.circle(frame, eye_center, radius, (255, 0, 0 ), 4)
    return frame


def show_img(title, img):
    """Display an image

    Args:
        title (str): The title of the display window
        img (Mat): The image to be displayed
    """
    # show image
    cv2.imshow(title, img)
    
    # wait indefinitly for a key press event
    keypressed = cv2.waitKey(0)
    # if the ESC key is pressed, destroy all windows and release resources
    if keypressed == 27:
        cv2.destroyAllWindows()

In [26]:
# detect faces
img_for_detection = cv2.imread("images/img-3.jpg")

# show result
show_img("Detected face", detect_face(img_for_detection))

#### PART 3 - WORKING WITH CAMERAS AND VIDEOS

In [28]:
# start video capture
capture = cv2.VideoCapture(0)

# check whether the capture was opened successfully
if not capture.isOpened():
    print("Unable to start camera")

In [29]:
while True:
    # capture the next frame
    ret, frame = capture.read()
    
    # check whether the frame is available or not
    if frame is None:
        break
    
    # show the captured frame
    cv2.imshow("Video", frame)
    
    # wait 30 milliseconds for a key press event
    keypressed = cv2.waitKey(30)
    # if the ESC key is pressed, destroy all windows and release resources
    if keypressed == 27:
        # release camera
        capture.release()
        # destroy all windows
        cv2.destroyAllWindows()
        break

#### PART 2 - OBJECT DETECTION IN VIDEOS

In [32]:
# start video capture
capture = cv2.VideoCapture(0)

# check whether the capture was opened successfully
if not capture.isOpened():
    print("Unable to start camera")

while True:
    # capture the next frame
    ret, frame = capture.read()
    
    # check whether the frame is available or not
    if frame is None:
        break
    
    # detect faces
    frame = detect_face_cuda(frame)
    
    # show the captured frame
    cv2.imshow("Video", frame)
    
    # wait 30 milliseconds for a key press event
    keypressed = cv2.waitKey(30)
    # if the ESC key is pressed, destroy all windows and release resources
    if keypressed == 27:
        # release camera
        capture.release()
        # destroy all windows
        cv2.destroyAllWindows()
        break